<a href="https://colab.research.google.com/github/takashisuo/nlp/blob/main/spam_filter_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import glob
import os
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
!pip install transformers
import torch
!pip install torchtext
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install tqdm
from tqdm import tqdm

     |████████████████████████████████| 2.2MB 20.8MB/s 
     |████████████████████████████████| 3.3MB 46.9MB/s 
     |████████████████████████████████| 870kB 44.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=4ddc048c6a3abd4eec7f95a72b4130576e4a56c4841d5ff6562366ee87df1b69
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested pac

In [ ]:
import torch
import torchtext

In [ ]:
ls "./drive/My Drive/Colab Notebooks/signate/spam/"

 sample_submit.csv        test_rev_sample.csv   train_rev.csv
 spam_filter_bert.ipynb   test_rev_test.csv     train_rev_test.csv
 test/                    test_rev.tsv         'train_rev - コピー.tsv'
 test_df.pickle           train/                train_rev.tsv
 test_rev.csv             train_df.pickle
 test_rev_noproblem.csv   train_master.tsv


In [ ]:
root = './drive/My Drive/Colab Notebooks/signate/spam'
train_df = pd.read_csv(root + "/train_master.tsv", delimiter='\t')

In [ ]:
train_df.head()

,file_name,label
0,train_0000.txt,0
1,train_0001.txt,0
2,train_0002.txt,1
3,train_0003.txt,1
4,train_0004.txt,0


In [ ]:
import pickle
train_pickle = root + '/train_df.pickle'
if os.path.exists(train_pickle):
  with open(train_pickle, 'rb') as web:
    train_df = pickle.load(web)
else:
  train_df["txt"] = 0
  i = 0
  train_files = sorted(glob.glob(root + "/train/*.txt"))
  for file in train_files:
      bfilename = os.path.basename(file)
      with open(file) as f:
          data = f.read()
          train_df.loc[i, "txt"] = data
          data = None
      i += 1
  print(train_df.head())
  with open(train_pickle, 'wb') as web:
    pickle.dump(train_df , web)

In [ ]:
# test
import pickle
from tqdm import tqdm
test_df = None
test_pickle = root + '/test_df.pickle'
if os.path.exists(test_pickle):
  with open(test_pickle, 'rb') as web:
    test_df = pickle.load(web)
else:
  test_files = sorted(glob.glob(root + "/test/*.txt"))
  # 以下のlabelは初期値なので意味なし
  test_df = pd.read_csv(root + "/sample_submit.csv", names=('file_name', 'label'))
  test_df["txt"] = 0
  i = 0
  for file in tqdm(test_files):
      bfilename = os.path.basename(file)
      with open(file) as f:
          data = f.read()
          test_df.loc[i, "txt"] = data
          data = None
      i += 1
  print(test_df.head())
  with open(test_pickle, 'wb') as web:
    pickle.dump(test_df , web)

In [ ]:
test_df.head(100)

,file_name,label,txt
0,test_0000.txt,1,Subject: join the thousands who are now sp @ m...
1,test_0001.txt,1,Subject: potential list fo 9 / 00\ndaren :\npe...
2,test_0002.txt,0,Subject: bounce skel @ iit . demokritos . gr :...
3,test_0003.txt,1,Subject: hpl meter # 981488 paris tenaska hpl\...
4,test_0004.txt,0,"Subject: hpl nom for august 3 , 2000\n( see at..."
...,...,...,...
95,test_0095.txt,0,Subject: re : october wellhead\nthe heatherloc...
96,test_0096.txt,1,Subject: watch this stock going up\na niche re...
97,test_0097.txt,1,"Subject: potential list for april , 2000\nthe ..."
98,test_0098.txt,0,Subject: re : tenaska iv\nany word yet ? i don...


In [ ]:
# preprocess method
def to_lower(word):
    result = word.lower()
    return result

def remove_special_characters(word):
    result = word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
    return result

def remove_stop_words(words):
    result = [i for i in words if i not in ENGLISH_STOP_WORDS]
    return ''.join(result)

def remove_hyperlink(word):
    return re.sub(r"http\S+", "", word)

def remove_linefeed(words):
  return re.sub(r"[\r\n]", " ", words)

In [ ]:
"""
test_files = glob.glob(root + "/test/*.txt")
test_df = pd.read_csv(root + "/sample_submit.csv", names=('file_name', 'label'))
test_df["txt"] = 0
i = 0
for file in test_files:
    bfilename = os.path.basename(file)
    if bfilename != 'test_0043.txt':
      i += 1
      continue
    print(bfilename)
    with open(file) as f:
        data = f.read()
        print(data)
        test_df.loc[i, "txt"] = data
        print(test_df.loc[i, "txt"])
    i += 1
test_df.query('file_name=="test_0043.txt"')
"""

'\ntest_files = glob.glob(root + "/test/*.txt")\ntest_df = pd.read_csv(root + "/sample_submit.csv", names=(\'file_name\', \'label\'))\ntest_df["txt"] = 0\ni = 0\nfor file in test_files:\n    bfilename = os.path.basename(file)\n    if bfilename != \'test_0043.txt\':\n      i += 1\n      continue\n    print(bfilename)\n    with open(file) as f:\n        data = f.read()\n        print(data)\n        test_df.loc[i, "txt"] = data\n        print(test_df.loc[i, "txt"])\n    i += 1\ntest_df.query(\'file_name=="test_0043.txt"\')\n'

In [ ]:
test_df.query('file_name=="test_0043.txt"')

,file_name,label,txt
43,test_0043.txt,1,Subject: aep transition items\nattached is a b...


In [ ]:
# 小文字へ置換する
train_df["txt"] = train_df["txt"].map(to_lower)
test_df["txt"] = test_df["txt"].map(to_lower)

In [ ]:
# 特殊文字の置換
train_df["txt"] = train_df["txt"].map(remove_special_characters)
test_df["txt"] = test_df["txt"].map(remove_special_characters)

In [ ]:
# ストップワード削除
train_df["txt"] = train_df["txt"].map(remove_stop_words)
test_df["txt"] = test_df["txt"].map(remove_stop_words)

In [ ]:
# URL削除
train_df["txt"] = train_df["txt"].map(remove_hyperlink)
test_df["txt"] = test_df["txt"].map(remove_hyperlink)

In [ ]:
# 改行削除
train_df["txt"] = train_df["txt"].map(remove_linefeed)
test_df["txt"] = test_df["txt"].map(remove_linefeed)

In [ ]:
import MeCab
def tokenize(text):
  tagger = MeCab.Tagger('-Owakati')
  sentence = tagger.parse(text)
  #sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
  sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
  return sentence.split()

In [ ]:

txt_list = []
for index, txt in enumerate(train_df["txt"]):
  txt_list.append(tokenize(txt))

train_df["tokenize"] = txt_list

txt_test_list = []
for index, txt in enumerate(test_df["txt"]):
  txt_test_list.append(tokenize(txt))
test_df["tokenize"] = txt_test_list

In [ ]:
test_df.head()

,label,txt
0,1,subject jon the thousnds who re now sp m fre...
1,1,subject potentl lst fo 9 00 dren per vnce s...
2,0,subject bounce skel t demokrtos gr non me...
3,1,subject hpl meter 981488 prs tensk hpl durng ...
4,0,subject hpl nom for ugust 3 2000 see ttched ...


In [ ]:
test_new_df = test_df["txt"].where(re.fullmatch('Subject  ', df["txt"]), "Subject spam")
test_new_df.query('index == "183"')

TypeError: ignored

In [ ]:
test_new_df.to_csv(root + '/test_rev_sample.csv', header=True, index=False)
# 一部の文章は過剰に削除されているようなので追加調査が必要

In [ ]:
"""
# トークン化
# https://lionbridge.ai/ja/articles/lstm-spam-mail-detection/
max_feature = 50000 #number of unique words to consider
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_feature)
tokenizer.fit_on_texts(train_df)
x_train_features = np.array(tokenizer.texts_to_sequences(x_train))
x_test_features = np.array(tokenizer.texts_to_sequences(x_test))
"""

'\n# トークン化\n# https://lionbridge.ai/ja/articles/lstm-spam-mail-detection/\nmax_feature = 50000 #number of unique words to consider\nfrom keras.preprocessing.text import Tokenizer\ntokenizer = Tokenizer(num_words=max_feature)\ntokenizer.fit_on_texts(train_df)\nx_train_features = np.array(tokenizer.texts_to_sequences(x_train))\nx_test_features = np.array(tokenizer.texts_to_sequences(x_test))\n'

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast

sc_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
sc_model.cuda()
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
import torchtext.data
max_length = 2500
#TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer, use_vocab=True,
#                           include_lengths=True, batch_first=True, fix_length=max_length)
TEXT = torchtext.legacy.data.Field(tokenize=tokenizer)
LABEL =torchtext.legacy.data.Field(sequential=False, use_vocab=False)

In [ ]:
print(train_df.head())
print(test_df.head())

   label                                                txt
0      0  subject re  buybck  defcency dels worksheet th...
1      0  subject fw  stress relef      orgnl messge    ...
2      1  subject from mrs  juln der frend  plese don  t...
3      1  subject  wrenches  68  off dremwever mx 2004 f...
4      0  subject y 2 k  texs log nme home pger george g...
   label                                                txt
0      1  subject jon the thousnds who re now sp  m  fre...
1      1  subject potentl lst fo 9  00 dren  per vnce  s...
2      0  subject bounce skel  t  demokrtos  gr  non  me...
3      1  subject hpl meter  981488 prs tensk hpl durng ...
4      0  subject hpl nom for ugust 3  2000  see ttched ...


In [ ]:
train_df = train_df.drop(["file_name", "tokenize"], axis=1)

KeyError: ignored

In [ ]:
test_df = test_df.drop(["file_name", "tokenize"], axis=1)

In [ ]:
test_df.iloc[0, 1]

'subject jon the thousnds who re now sp  m  free forget spm blockers  get smrt spm control tht lwys delvers the eml you wnt  fnlly  we dscovered the ultmte soluton tht s gurnteed to stop ll spm wthout losng ny of your mportnt eml  ths revolutonry dvnced technology lso protects you 100  gnst ll eml  borne vruses  both known nd unknown  we ddn  t beleve t ether untl we ctully tred t  so you be the judge nd see for yourself  clck here nd lern more you owe t to yourself to try ths progrm  nd forwrd ths eml to ll of your frends who hte spm or s mny people s possble  we re strongly gnst sendng unsolcted emls to those who do not wsh to receve our specl mlngs  you hve opted n to one or more of our fflte stes requestng to be notfed of ny specl offers we my run from tme to tme  we lso hve ttned the servces of n ndependent 3 rd prty to overlook lst mngement nd removl servces  ths s not unsolcted eml  f you do not wsh to receve further mlngs  plese clck here to be removed from the lst  plese ccept

In [ ]:
# test_dfのsubject単体の箇所が問題、そこを修正すれば対応可能。ただし
train_df.to_csv(root + '/train_rev.csv', header=True, index=False)
test_new_df.to_csv(root + '/test_rev.csv', header=True, index=False)

In [ ]:
path = root + '/train_rev.csv'
train_ds = torchtext.legacy.data.TabularDataset(path=path, format='csv',  
        fields=[('label', LABEL), ('txt', TEXT)])

path = root + '/test_rev.csv'
test_ds = torchtext.legacy.data.TabularDataset(path=path, format='csv',  
        fields=[('label', LABEL), ('txt', TEXT)])

In [ ]:
train, test = torchtext.legacy.data.TabularDataset.splits(
        path=root,
        train='train_rev.csv',
        test='test_rev.csv',
        format='csv',
        fields=[('label', LABEL), ('text', TEXT)])

In [ ]:
df = pd.read_csv(root + '/test_rev.csv')
df.info

<bound method DataFrame.info of              label           txt
0     Subject spam  Subject spam
1     Subject spam  Subject spam
2     Subject spam  Subject spam
3     Subject spam  Subject spam
4     Subject spam  Subject spam
...            ...           ...
2581  Subject spam  Subject spam
2582  Subject spam  Subject spam
2583  Subject spam  Subject spam
2584  Subject spam  Subject spam
2585  Subject spam  Subject spam

[2586 rows x 2 columns]>

In [ ]:
from torchtext.data import Field, Dataset, Example
import pandas as pd

class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
            examples pd.DataFrame: DataFrame of examples
            fields {str: Field}: The Fields to use in this tuple. The
                string is a field name, and the Field is the associated field.
            filter_pred (callable or None): use only exanples for which
                filter_pred(example) is true, or use all examples if None.
                Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""
  
    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()
        
        for key, field in fields.items():
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex

ImportError: ignored

In [ ]:
train_data = torch.tensor(train_df.drop(['label', 'file_name'], axis = 1).values.)
display(train_data)

TypeError: ignored

In [ ]:
train_label = torch.tensor(train_df['txt'].values)

In [ ]:
# for test
import pandas as pd
import torch
import random

# creating dummy targets (float values)
targets_data = [random.random() for i in range(10)]

# creating DataFrame from targets_data
targets_df = pd.DataFrame(data=targets_data)
targets_df.columns = ['targets']
display(targets_df.head())

# creating tensor from targets_df 
torch_tensor = torch.tensor(targets_df['targets'].values)

# printing out result
print(torch_tensor)

,targets
0,0.810790
1,0.650089
2,0.496801
3,0.149310
4,0.819875


tensor([0.8108, 0.6501, 0.4968, 0.1493, 0.8199, 0.7657, 0.4379, 0.6082, 0.1631,
        0.4082], dtype=torch.float64)


In [ ]:
train_df.head()

,file_name,label,txt
0,train_0000.txt,0,subject from brnd nmes to genercs from overex...
1,train_0001.txt,0,subject hpl nom for februry 13 2001\n see ttc...
2,train_0002.txt,1,subject chepest drug s on the net gurnteed...
3,train_0003.txt,1,subject re deer prk rlrod crossng 18 3026 ln...
4,train_0004.txt,0,subject bndstnd\ngenerc vgr t 1 63\nno presc...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(train_df['txt'])

print('Vocabulary size: {}'.format(len(vectorizer.vocabulary_)))
print('Vocabulary content: {}'.format(type(vectorizer.vocabulary_)))

Vocabulary size: 31279
Vocabulary content: <class 'dict'>


In [ ]:
# 文章を特徴ベクトル化
X_train_bow = vectorizer.transform(train_df['txt'])
X_test_bow = vectorizer.transform(test_df['txt'])

print('X_train_bow:\n{}'.format(repr(X_train_bow)))
print('X_test_bow:\n{}'.format(repr(X_test_bow)))

X_train_bow:
<2586x31279 sparse matrix of type '<class 'numpy.float64'>'
	with 220992 stored elements in Compressed Sparse Row format>
X_test_bow:
<2586x31279 sparse matrix of type '<class 'numpy.float64'>'
	with 199159 stored elements in Compressed Sparse Row format>


In [ ]:
print(len(vectorizer.vocabulary_))
#print(dict(list(vectorizer.vocabulary_.items())))
#print(sorted(dict(list(vectorizer.vocabulary_.items()))), key=lambda x:x[0])

31279


In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_bow, train_df.loc[:,"label"])
print('Train accuracy: {:.3f}'.format(model.score(X_train_bow, train_df.loc[:,"label"])))

Train accuracy: 0.915


In [ ]:
pred = model.predict(X_test_bow)

In [ ]:
test_files = glob.glob("./test/*.txt")
submit_df = pd.DataFrame(columns=["file", "label"])
for i, file in enumerate(test_files):
    tmp_se = pd.Series([os.path.basename(file), pred[i]])
    submit_df.loc[i, "file"] = os.path.basename(file)
    submit_df.loc[i, "label"] = pred[i]
submit_df.head()

,file,label
0,test_0000.txt,1
1,test_0001.txt,0
2,test_0002.txt,1
3,test_0003.txt,0
4,test_0004.txt,0


In [ ]:
submit_df.to_csv("./submit_tfidf_MultinomialNB_preproc.csv", header=None, index=0)